<h1>Módulo de Extracción de datos</h1>

Este notebook tiene como finalidad realizar una recabación de twits relacionados al mundial 2022. Para realizar esta extracción de tweets utilizaremos las siguientes herramientas: Tweepy y Pandas.

In [ ]:
import tweepy
import time
import pandas as pd
pd.set_option('display.max_colwidth', None)
import re

In [ ]:
%run ./twitter_keys.ipynb

In [ ]:
#Online authentication. Proveemos nuestro consumer_key y secret
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
#Proveemos nuestro access token y secret.
auth.set_access_token(access_token,access_token_secret)
api = tweepy.API(auth)

In [ ]:
#contenido del tweet
tweets = []
#cantidad de likes del tweet
likes = []
#tiempo en el cual el tweet se creo
time = []

In [ ]:
query = '#MundialQatar2022 OR Qatar OR Mundial OR Catar 2022 OR #Qatar2022 -filter:retweets lang:es'


In [ ]:
for tweet in tweepy.Cursor(api.search_tweets, 
                                 q = query).items(1500):
    text = tweet.text.replace('\n',' ')
    clean_tweet = re.sub("@[A-Za-z0-9]+","", text)
    clean_tweet = re.sub("#[A-Za-z0-9]+","", clean_tweet)
    clean_tweet = re.sub(r"http\S+", "", clean_tweet)
    clean_tweet = re.sub(r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", clean_tweet)
    tweets.append(clean_tweet.lower())
    likes.append(tweet.favorite_count)
    time.append(tweet.created_at)

In [ ]:
df2 = pd.DataFrame({'tweets':tweets,'likes':likes,'time':time})

In [ ]:
df2

In [ ]:
df2.head(500)

In [ ]:
from datetime import datetime
now = datetime.now()
df.to_csv('D:\\FACU\\TF_CSVs\\'+'WorldCupTweets_' + str(now)[:16].replace(':','_') + '.csv',
          index=True,
          sep=';'
         )
